<a href="https://colab.research.google.com/github/FarrazNouval/Python-for-business-administration/blob/main/FIFO_costing_for_warehousing_dept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFO costing

Inventory valuation is a periodic activity in the series of material costing activities. It is the final action in a series of material costing tasks and is sometimes difficult due to the vast amount of data. In this notebook, I'll write a program to perform inventory valuation using the FIFO method to handle large amounts of data automatically.

FIFO is an inventory valuation method that stands for First In, First Out, where goods acquired or produced first are assumed to be sold first. This means that when a business calculates its cost of goods sold for a given period, it uses the costs from the oldest inventory assets.

In some cases, a business may not actually sell or dispose of its oldest goods first. However, FIFO assumes that it does so for the sake of accounting.

There are various approaches or procedures for determining the cost of materials used to manufacture a product in inventory valuation; however, in this notebook, I will be using the FIFO approach.

The dataset used in this notebook is not a real data, but i tried to make it as similar as possible to the factual dataset for inventory valuation.

* The price assumed in this dataset includes commercial and cash discounts, tax, shipping, and so on (price when material arrives in the warehouse).

Why should you use the FIFO method instead of the other methods?
1. Simplicity: It's a straightforward method to understand and implement. Assuming a consistent flow of inventory, it's relatively easy to calculate the COGS and ending inventory.
2. Relevance in Periods of Falling Prices: When prices are falling, FIFO tends to result in a lower COGS and a higher gross profit. This is because the older, more expensive units are assumed to be sold first, leaving the cheaper units in inventory. This can be beneficial for businesses during periods of deflation.
3. Matching Costs to Revenues: FIFO aligns with the principle of matching costs to revenues, as it assumes that the costs incurred to acquire the inventory sold are matched against the revenue generated from that sale. This can provide a more accurate representation of a company's profitability.

Why using the other methods instead of the FIFO method?
1. Inaccurate Cost of Goods Sold (COGS) in Periods of Rising Prices: When prices are rising, the FIFO method can result in an understated COGS and, consequently, an overstated gross profit. This is because the older, cheaper units are assumed to be sold first, leaving the more expensive units in inventory. This can distort the financial picture of a company.
2. Tax Implications: In periods of rising prices, the FIFO method can lead to higher taxable income due to the overstated gross profit. This can result in higher tax liabilities.
3. Inventory Valuation Issues: The FIFO method assumes that the first units purchased are the first units sold. However, in reality, this may not always be the case, especially if there is product obsolescence or damage. This can lead to inaccurate inventory valuations.

steps to do:
1. get the required packages
2. load the dataset
3. check the dataset
4. preprocessing the dataset
5. building the system to solve the problem
6. testing

In [1]:
# get the required packages
import pandas as pd
import os
import plotly.express as px

In [2]:
# load dataframe
df = pd.read_csv("/content/material_costing.csv")
df


,date,status,inventory code,quantity,price,total
0,01/01/2023,in,INV-123456,500,25000.0,12500000.0
1,01/01/2023,in,INV-123457,200,50000.0,10000000.0
2,01/01/2023,in,INV-123458,300,60000.0,18000000.0
3,01/01/2023,in,INV-123459,500,19000.0,9500000.0
4,01/01/2023,in,INV-123460,300,20000.0,6000000.0
5,02/01/2023,in,INV-123457,100,45000.0,4500000.0
6,05/01/2023,out,INV-123458,150,NaN,NaN
7,05/01/2023,out,INV-123460,150,NaN,NaN
8,10/01/2023,out,INV-123456,200,NaN,NaN
9,10/01/2023,out,INV-123457,100,NaN,NaN


In [3]:
# check the dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            14 non-null     object 
 1   status          14 non-null     object 
 2   inventory code  14 non-null     object 
 3   quantity        14 non-null     int64  
 4   price           7 non-null      float64
 5   total           7 non-null      float64
dtypes: float64(2), int64(1), object(3)
memory usage: 800.0+ bytes


In [4]:
# transform date dtype to datetime
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            14 non-null     datetime64[ns]
 1   status          14 non-null     object        
 2   inventory code  14 non-null     object        
 3   quantity        14 non-null     int64         
 4   price           7 non-null      float64       
 5   total           7 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 800.0+ bytes


In [5]:
# class for material
class Material:
    def __init__(self, date, inventory, inventory_code, quantity, price):
        self.date = date
        self.inventory = inventory
        self.inventory_code = inventory_code
        self.quantity = quantity
        self.price = price
        self.total = quantity * price

In [6]:
# class for process
class InventoryFIFO:
    def __init__(self, df):
        self.inventory = {}
        self.out_material = {}
        self.df = df

    # filtering data by date
    def filter_data(self, date_from, date_to):
        date_from = pd.to_datetime(date_from, format="%Y/%m/%d")
        date_to = pd.to_datetime(date_to, format="%Y/%m/%d")

        self.df = self.df[(self.df["date"] >= date_from) & (self.df["date"] <= date_to)]
        self.df = self.df.sort_values(by="date", ascending=True)

    # function to add material to the inventory
    def add_material(self, material, status):
        if status == "in":
            if material.inventory_code not in self.inventory:
                self.inventory[material.inventory_code] = []
            self.inventory[material.inventory_code].append(material)
        else:
            if material.inventory_code not in self.out_material:
                self.out_material[material.inventory_code] = []
            self.out_material[material.inventory_code].append(material)

    # function to extracting data from the source and inputting it to the system
    def input_data(self):
        for idx, row in self.df.iterrows():
            if row['status'] == "in":
                self.add_material(Material(row['date'], row['status'], row['inventory code'], row['quantity'], row['price']), "in")
            elif row['status'] == "out":
                cost, minus_quantity = self.remove_material(row['inventory code'], row['quantity'], row['date'])
            elif row['status'] == "return":
                self.return_material(Material(row['date'], row['status'], row['inventory code'], row['quantity'], row['price']))

    # function to remove the material from inventory to the manufacture process
    def remove_material(self, inventory_code, quantity, date):
        if inventory_code not in self.inventory or not self.inventory[inventory_code]:
            raise ValueError("No inventory available for the given code")

        total_material_quantity = 0
        for i in self.inventory[inventory_code]:
            total_material_quantity += i.quantity

        total_cost = 0
        minus_quantity = 0
        out_quantity = 0
        while quantity > 0 and self.inventory[inventory_code]:
            for i in range(len(self.inventory[inventory_code])):
                current_material = self.inventory[inventory_code][i]
                if total_material_quantity >= quantity:
                    if current_material.quantity < quantity:
                        quantity -= current_material.quantity
                        out_quantity = current_material.quantity
                        current_material.quantity = 0
                        total_cost += current_material.total
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")

                    else:
                        current_material.quantity -= quantity
                        total_cost += quantity * current_material.price
                        current_material.total = current_material.quantity * current_material.price
                        out_quantity = quantity
                        quantity = 0
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")
                        break
                # if the current total material quantity is less than the required quantity
                else:
                    if current_material.quantity < quantity:
                        out_quantity += quantity
                        quantity -= current_material.quantity
                        minus_quantity += quantity
                        current_material.quantity = 0
                        total_cost += current_material.total
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")


        return total_cost, minus_quantity

    # function to return material from processing plant to the inventory
    def return_material(self, material):
        returned_material = self.out_material[material.inventory_code][-1]
        self.add_material(Material(material.date,
                                   material.inventory,
                                   material.inventory_code, material.quantity,
                                   returned_material.price), "in")

    # function to get remaining materials in the inventory
    def get_inventory(self, inventory_code):
        return list(self.inventory.get(inventory_code, []))

    # function to get the used materials
    def get_removed(self, inventory_code):
        return list(self.out_material.get(inventory_code, []))

    # function to simplify material structure
    def simplify_material(self, material):
        new_material = {}
        for inv_code in material.keys():
            for inv in material[inv_code]:
                date = inv.date
                status = inv.inventory
                invent_code = inv.inventory_code
                qty = inv.quantity
                price = inv.price
                total = inv.total

                data = [date, status, invent_code, qty, price, total]

                if inv_code not in new_material.keys():
                    new_material[inv_code] = []
                    new_material[inv_code].append(data)
                else:
                    new_material[inv_code].append(data)

        return new_material

    # function to store materials info in a pandas dataframe
    def to_material_df(self, simplified_material, show_zeros):
        new_material = {"date":[],
                        "status":[],
                        "inventory code":[],
                        "quantity":[],
                        "price":[]}

        for inv_code in simplified_material.keys():
            for item in simplified_material[inv_code]:
                new_material["date"].append(item[0])
                new_material["status"].append(item[1])
                new_material["inventory code"].append(item[2])
                new_material["quantity"].append(item[3])
                new_material["price"].append(item[4])

        material_df = pd.DataFrame(new_material)
        material_df.sort_values(by="date", inplace=True)
        material_df.reset_index(inplace=True)
        del material_df['index']
        material_df["total"] = material_df["price"] * material_df["quantity"]

        if not show_zeros:
            material_df = material_df[material_df["quantity"] > 0]

        return material_df


In [7]:
# make an object of inventory
inv_fifo = InventoryFIFO(df)

print("enter date to filter")
date_from = input("start date, format = yyyy/mm/dd => 2022/07/22")
date_to = input("end date, format = yyyy/mm/dd => 2022/07/22")
show_zeros = input("do you want to show material with zero quantity or not, Y/N")

inv_fifo.filter_data(date_from, date_to)
inv_fifo.input_data()

remaining_inventory = inv_fifo.inventory
out_inventory = inv_fifo.out_material

simple_remaining_material = inv_fifo.simplify_material(remaining_inventory)
simple_out_material = inv_fifo.simplify_material(out_inventory)

if show_zeros.lower() == "y":
    remaining_material_df = inv_fifo.to_material_df(simple_remaining_material, True)
    out_material_df = inv_fifo.to_material_df(simple_out_material, True)
else:
    remaining_material_df = inv_fifo.to_material_df(simple_remaining_material, False)
    out_material_df = inv_fifo.to_material_df(simple_out_material, False)


enter date to filter
start date, format = yyyy/mm/dd => 2022/07/222023/01/01
end date, format = yyyy/mm/dd => 2022/07/222023/01/20
do you want to show material with zero quantity or not, Y/Ny


In [8]:
remaining_material_df


,date,status,inventory code,quantity,price,total
0,2023-01-01,in,INV-123456,300,25000.0,7500000.0
1,2023-01-01,in,INV-123457,0,50000.0,0.0
2,2023-01-01,in,INV-123458,150,60000.0,9000000.0
3,2023-01-01,in,INV-123459,300,19000.0,5700000.0
4,2023-01-01,in,INV-123460,150,20000.0,3000000.0
5,2023-01-02,in,INV-123457,50,45000.0,2250000.0
6,2023-01-15,in,INV-123457,200,45500.0,9100000.0
7,2023-01-20,return,INV-123457,100,45000.0,4500000.0


In [9]:
out_material_df


,date,status,inventory code,quantity,price,total
0,2023-01-05,out,INV-123458,150,60000.0,9000000.0
1,2023-01-05,out,INV-123460,150,20000.0,3000000.0
2,2023-01-10,out,INV-123456,200,25000.0,5000000.0
3,2023-01-10,out,INV-123457,100,50000.0,5000000.0
4,2023-01-11,out,INV-123457,100,50000.0,5000000.0
5,2023-01-11,out,INV-123457,50,45000.0,2250000.0
6,2023-01-15,out,INV-123459,200,19000.0,3800000.0


In [10]:
# total cost
total_material_cost = out_material_df.total.sum()
total_material_cost


33050000.0

In [11]:
# total remaining inventory
total_remaining_material = remaining_material_df.total.sum()
total_remaining_material


41050000.0

assuming the company produces 150000 units of product x from 2020/01/01 to 2020/01/20, how much is the material cost of product x per unit?

In [12]:
unit_produced = 150000
unit_cost = round(total_material_cost / unit_produced, 2)
print(f"unit cost : {unit_cost}")


unit cost : 220.33


In [13]:
used_material = out_material_df.groupby("inventory code").agg({"quantity":"sum",
                                                               "total":"sum"}).reset_index()
used_material["price per unit"] = used_material["total"] / used_material["quantity"]
used_material


,inventory code,quantity,total,price per unit
0,INV-123456,200,5000000.0,25000.0
1,INV-123457,250,12250000.0,49000.0
2,INV-123458,150,9000000.0,60000.0
3,INV-123459,200,3800000.0,19000.0
4,INV-123460,150,3000000.0,20000.0


### How much material was used to produce product X from January 1st to January 20th, 2023 for each inventory code?
To answer this question, we can create a bar chart that shows how much material is used for each inventory code.

In [17]:
fig = px.histogram(used_material,
                   x="inventory code",
                   y="quantity",
                   color="inventory code",
                   title="Material was used to produce product X from January 1st to January 20th, 2023",
                   text_auto=True)
fig.show()


### What is the total cost of materials used to produce product X from January 1st to January 20th, 2023, for each inventory code?
To answer this question, we can make a bar chart that shows how much total material cost is spent for each inventory code.

In [15]:
fig = px.histogram(used_material,
                   x="inventory code",
                   y="total",
                   color="inventory code",
                   title="Price of materials used to produce product x from 1st to 20th january",
                   text_auto=True)
fig.show()


### What is the cost per unit of materials used to produce product X from January 1st to January 20th, 2023, for each inventory code?
To answer this question, we can make a bar chart that shows the material cost per unit for each inventory code.


In [18]:
fig = px.histogram(used_material,
                   x="inventory code",
                   y="price per unit",
                   color="inventory code",
                   title="Material cost per unit used to produce product x from 1st to 20th january",
                   text_auto=True)
fig.show()


## conclusion
Based on the analysis, we can conclude that:
1. The total material cost to produce product x from 2023/01/01 to 2023/01/20 is 33.050.000.
2. The total remaining material values in the inventory at 2023/01/20 after production are 41.100.000.
3. The unit cost of product x is 220.33.
4. we can use the price per unit of used material to negotiate the better material price with suppliers.

The result can be used to evaluate:
1. the supply chain and manufacturing process,
2. assess the effectiveness of the production plan,
3. establish the efficiency of the manufacturing process,
4. decide pricing strategy.

Furthermore, the company can use it to optimize its production process to reduce costs and increase profits by negotiating better prices with suppliers.

## reference
1. Dr. Murtadho al Qasim, cost accounting (arabic book)
2. [freshbooks.com, What Is FIFO Method: Definition and Guide](https://www.freshbooks.com/hub/accounting/what-is-fifo?srsltid=AfmBOorAZVOqOCqOgoWay4XoNp3qg7K8ilxiXnR-mAR7npbEUfmg8sfq)
3. Gemini AI